# Trends and change points in fire weather and drought

In [ ]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client

On Pearcey, nodes have 20 cores, each with 6GB. More than 20 cores, need more than one worker. `processes` ensures the number of workers.

In [ ]:
cluster = SLURMCluster(#processes=1,
                       walltime='03:00:00',
                       cores=20,
                       memory='120GB',
                       job_extra=['--qos="express"'])

In [ ]:
cluster.scale(jobs=1)

In [ ]:
client = Client(cluster)
client

In [ ]:
client.close()
cluster.close()

In [ ]:
import xarray as xr
import numpy as np
import string
import copy

import matplotlib.pyplot as plt
import matplotlib as mpl

import cartopy
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
cartopy.config['pre_existing_data_dir'] = '/datasets/work/oa-dcfp/work/squ027/data/cartopy-data'
cartopy.config['data_dir'] = '/datasets/work/oa-dcfp/work/squ027/data/cartopy-data'

In [ ]:
import functions as fn

# Plotting parameters

In [ ]:
fontsize = 7
coastlines_lw = 0.5
linewidth = 1.1
patheffect_lw_add = linewidth * 1.8
    
plt_params = {'lines.linewidth': linewidth,
              
              'hatch.linewidth': 0.5,
    
              'font.size': fontsize,
              
              'legend.fontsize' : fontsize-1,
              'legend.columnspacing': 0.7,
              'legend.labelspacing' : 0.03,
              'legend.handlelength' : 1.,
             
              'axes.linewidth': 0.5}

# default colours
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

letters = list(string.ascii_lowercase)

# Custom cmap
custom_cmap = fn.get_magma_waterlily_cmap()

# Load masks

### JRA land mask

In [ ]:
jra_mask = xr.open_zarr('/scratch1/ric368/projects/fire/data/masks/jra_land_mask.zarr',
                        consolidated=True).land
jra_mask = jra_mask.compute()

### Burned area mask on JRA grid

In [ ]:
jra_ba_mask = xr.open_zarr('/scratch1/ric368/projects/fire/data/masks/jra_burned_area_mask.zarr',
                            consolidated=True).burned_cells
jra_ba_mask = jra_ba_mask.compute()

# Load FFDI and precipitation data

In [ ]:
# Extreme FFDI days per year
grid_ex_dpy = xr.open_zarr('/scratch1/ric368/projects/fire/data/derived/jra_extreme_ffdi_dpy_global_1959_2020.zarr',
                             consolidated=True).ffdi_dpy
grid_ex_dpy = grid_ex_dpy.compute()

In [ ]:
# Precipitation for fire year
grid_ypr = xr.open_zarr('/scratch1/ric368/projects/fire/data/derived/gpcc_fire_year_precip_jra_grid_global_1959_2020.zarr',
                         consolidated=True).precip
grid_ypr = grid_ypr.sel(time=slice('1960', '2020')).compute()

# Set period and number of bootstraps

In [ ]:
reps = 1000
fy = 1970
ly = 2020

In [ ]:
grid_ex_dpy = grid_ex_dpy.sel(time=slice(str(fy), str(ly)))
grid_ypr = grid_ypr.sel(time=slice(str(fy), str(ly)))

In [ ]:
# Anomalise before trend and change point testing to remove spatial variability
grid_ypr = (grid_ypr - grid_ypr.mean('time')) / grid_ypr.std('time')

# Estimate block length for resampling

In [ ]:
ffdi_L_da = fn.estimate_L(grid_ex_dpy.where(jra_mask == 1))
gpcc_L_da = fn.estimate_L(grid_ypr.where(jra_mask == 1))

In [ ]:
# Range of block lengths needed
ffdi_L_range = np.unique(ffdi_L_da)
ffdi_L_range = ffdi_L_range[~np.isnan(ffdi_L_range)].astype('int16')
ffdi_L_da = ffdi_L_da.rename('L')

gpcc_L_range = np.unique(gpcc_L_da)
gpcc_L_range = gpcc_L_range[~np.isnan(gpcc_L_range)].astype('int16')
gpcc_L_da = gpcc_L_da.rename('L')

max_L = np.max([np.max(np.abs(ffdi_L_range)), np.max(np.abs(gpcc_L_range))])
max_L

# Figure S2: estimated block lengths for resampling

In [ ]:
with plt.rc_context(plt_params):
    fig, ax = plt.subplots(1, 2, figsize = (6.9, 2), subplot_kw={'projection': ccrs.PlateCarree()})
    
    ax[0].coastlines(color='black', lw=coastlines_lw)
    ffdi_L_da.plot(ax=ax[0], levels=range(max_L+1), add_colorbar=False, rasterized=True)
    ax[0].set_title('')
    ax[0].text(0.03, 0.2, 'Fire weather\ndays', transform=ax[0].transAxes)

    ax[1].coastlines(color='black', lw=coastlines_lw)
    p = gpcc_L_da.plot(ax=ax[1], levels=range(max_L+1), add_colorbar=False, rasterized=True)
    ax[1].set_title('')
    ax[1].text(0.03, 0.2, 'Precipitation', transform=ax[1].transAxes)
    
    cb_ax = fig.add_axes([0.21, 0.1, 0.6, 0.03])
    cb = fig.colorbar(p, cax=cb_ax, orientation='horizontal', ticks=np.arange(0.5,max_L+1.5))
    cb.ax.set_xticklabels(range(1,max_L+1))
    cb.ax.set_xlabel('Block length [years]')
    
    plt.subplots_adjust(wspace=0.01)
    
    plt.savefig('./figures/block_lengths_'+str(fy)+'-'+str(ly)+'.pdf', bbox_inches='tight', format='pdf', dpi=400)

# Pettitt test

In [ ]:
compute = True

### FFDI

#### Takes ~1.5 hours with 3 workers, each with 10 cores, 60GB

In [ ]:
%%time
if compute:
    # Pettitt test for each block length
    pettitt_ds_list = []
    for L in ffdi_L_range:
        print('Computing for L =', L)
        da = grid_ex_dpy.where(ffdi_L_da == L)
        pettitt_ds_list.append(fn.cp_map_pettitt_ufunc(da, reps, L, fy, ly).assign_coords({'L': L}))
        
    pettitt_ds = xr.concat(pettitt_ds_list, dim='L')
    pettitt_ds.to_zarr('/scratch1/ric368/projects/fire/data/derived/jra_ffdi_global_pettitt_'+str(reps)+'_reps_'+str(fy)+'-'+str(ly)+'.zarr',
                   mode='w',
                   consolidated=True)

In [ ]:
ffdi_pettitt_ds = xr.open_zarr('/scratch1/ric368/projects/fire/data/derived/jra_ffdi_global_pettitt_'+str(reps)+'_reps_'+str(fy)+'-'+str(ly)+'.zarr',
                               consolidated=True)

### Precipitation

#### Takes ~1 hour with 3 workers, each with 10 cores, 60GB

In [ ]:
%%time
if compute:
    pettitt_ds_list = []
    for L in gpcc_L_range:
        print('Computing for L =', L)
        da = grid_ypr.where(gpcc_L_da == L)
        pettitt_ds_list.append(fn.cp_map_pettitt_ufunc(da, reps, L, fy, ly).assign_coords({'L': L}))
        
    pettitt_ds = xr.concat(pettitt_ds_list, dim='L')
    pettitt_ds.to_zarr('/scratch1/ric368/projects/fire/data/derived/gpcc_precip_anoms_global_pettitt_'+str(reps)+'_reps_'+str(fy)+'-'+str(ly)+'.zarr',
                   mode='w',
                   consolidated=True)

In [ ]:
gpcc_pettitt_ds = xr.open_zarr('/scratch1/ric368/projects/fire/data/derived/gpcc_precip_anoms_global_pettitt_'+str(reps)+'_reps_'+str(fy)+'-'+str(ly)+'.zarr',
                               consolidated=True)

# Mann-Kendall trend test

For a time series $\{x_{t}, t=1, 2, \ldots, n\}$, the test statistic $S$ is given by:

$$S = \sum_{i=1}^{n-1} \sum_{j=i+1}^{n} \mathrm{sgn}(x_{j} - x_{i})$$

### FFDI

#### Takes ~ 1.5 hours with 1 workers, each 20 cores, 120GB

In [ ]:
%%time
if compute:
    mk_ds_list = []
    for L in ffdi_L_range:
        print('Computing for L =', L)
        da = grid_ex_dpy.where(ffdi_L_da == L)
        mk_ds_list.append(fn.cp_map_mk_ufunc(da, reps, L, fy, ly).assign_coords({'L': L}))
        
    mk_ds = xr.concat(mk_ds_list, dim='L')
    mk_ds.to_zarr('/scratch1/ric368/projects/fire/data/derived/jra_ffdi_global_mann_kendall_'+str(reps)+'_reps_'+str(fy)+'-'+str(ly)+'.zarr',
              mode='w',
              consolidated=True)

In [ ]:
ffdi_mk_ds = xr.open_zarr('/scratch1/ric368/projects/fire/data/derived/jra_ffdi_global_mann_kendall_'+str(reps)+'_reps_'+str(fy)+'-'+str(ly)+'.zarr',
                          consolidated=True)

### Precipitation

#### Takes ~ 1 hour with 1 workers, each 10 cores, 60GB

In [ ]:
%%time
if compute:
    mk_ds_list = []
    for L in gpcc_L_range:
        print('Computing for L =', L)
        da = grid_ypr.where(gpcc_L_da == L)
        mk_ds_list.append(fn.cp_map_mk_ufunc(da, reps, L, fy, ly).assign_coords({'L': L}))
        
    mk_ds = xr.concat(mk_ds_list, dim='L')
    mk_ds.to_zarr('/scratch1/ric368/projects/fire/data/derived/gpcc_precip_anoms_global_mann_kendall_'+str(reps)+'_reps_'+str(fy)+'-'+str(ly)+'.zarr',
              mode='w',
              consolidated=True)

In [ ]:
gpcc_mk_ds = xr.open_zarr('/scratch1/ric368/projects/fire/data/derived/gpcc_precip_anoms_global_mann_kendall_'+str(reps)+'_reps_'+str(fy)+'-'+str(ly)+'.zarr',
                          consolidated=True)

# Aggregate results for all $L$

In [ ]:
# ffdi_pettitt_all = ffdi_pettitt_ds.sum(dim='L').where(jra_ba_mask == 1).compute()
ffdi_mk_all = ffdi_mk_ds.sum(dim='L').where(jra_ba_mask == 1).compute()

In [ ]:
gpcc_pettitt_all = gpcc_pettitt_ds.sum(dim='L').where(jra_ba_mask == 1).compute()
gpcc_mk_all = gpcc_mk_ds.sum(dim='L').where(jra_ba_mask == 1).compute()

# False discovery rate

In [ ]:
# ffdi_pettitt_all['fdr_10'] = fn.fdr(ffdi_pettitt_all.p_value)
# gpcc_pettitt_all['fdr_10'] = fn.fdr(gpcc_pettitt_all.p_value)

ffdi_mk_all['fdr_10'] = fn.fdr(ffdi_mk_all.p_value)
# gpcc_mk_all['fdr_10'] = fn.fdr(gpcc_mk_all.p_value)

# Figure 4: FFDI change point and trend, and GPCC trend

In [ ]:
figsize = (6.9,3.)

# Colormap stuff
rgb_val = 0.9
bad_col = np.repeat(rgb_val, 3)

ffdi_cmap = copy.copy(custom_cmap)
ffdi_cmap.set_under(bad_col)

pr_cmap = copy.copy(mpl.cm.get_cmap('BrBG'))
pr_cmap.set_under(bad_col)

# For change point years, use discrete colormap
# select desired starting colours
colours = np.asarray(plt.rcParams['axes.prop_cycle'].by_key()['color'])[[7,4,0,1,2,6]]
# get three grads of lightness
l_colours = [fn.adjust_lightness(c, a) for c in colours for a in [1.3, 0.9]]
cmap = mpl.colors.ListedColormap(l_colours)
norm = mpl.colors.BoundaryNorm(np.arange(12.01), cmap.N) 
idx = [11,0,1,2,3,6,7,4,5,9]
l_colours = [l_colours[lc] for lc in idx]
yr_cmap = mpl.colors.ListedColormap(l_colours)

with mpl.rc_context(plt_params):
    ffdi_lon = ffdi_pettitt_all.lon
    ffdi_lat = ffdi_pettitt_all.lat
    
    gpcc_lon = gpcc_pettitt_all.lon
    gpcc_lat = gpcc_pettitt_all.lat
    
    fig, ax = plt.subplots(2, 2, figsize=figsize, subplot_kw={'projection': ccrs.PlateCarree()})

    # ================ Precip trend
    mags = xr.where(gpcc_mk_all.ts_slope.isnull(), -999, gpcc_mk_all.ts_slope).values
    cyclic_mags, cyclic_lon = add_cyclic_point(mags, coord=gpcc_lon)
    # First plot without set_under so that the colorbar doesn't register it
    p_pr_t = ax[0,0].pcolormesh(cyclic_lon, gpcc_lat, cyclic_mags, cmap='BrBG', zorder=1, vmin=-0.05, vmax=0.05, rasterized=True)
    # Then plot the colors we want
    ax[0,0].pcolormesh(cyclic_lon, gpcc_lat, cyclic_mags, cmap=pr_cmap, zorder=2, vmin=-0.05, vmax=0.05, rasterized=True)
    
    fdr = gpcc_mk_all.fdr_10
    fdr = xr.where(fdr == 1, 1, np.nan).values
    cyclic_fdr, cyclic_lon = add_cyclic_point(fdr, coord=gpcc_lon)
    ax[0,0].pcolor(cyclic_lon, gpcc_lat, cyclic_fdr, hatch='//////', shading='auto', alpha=0, rasterized=False, zorder=3)
    
    # ================ FFDI trend
    mags = xr.where(ffdi_mk_all.ts_slope.isnull(), -999, ffdi_mk_all.ts_slope).values
    cyclic_mags, cyclic_lon = add_cyclic_point(mags, coord=ffdi_lon)
    p_ffdi_t = ax[0,1].pcolormesh(cyclic_lon, ffdi_lat, cyclic_mags, cmap=ffdi_cmap, zorder=1, vmin=-0.9, vmax=0.9, rasterized=True)
    
    fdr = ffdi_mk_all.fdr_10
    fdr = xr.where(fdr == 1, 1, np.nan).values
    cyclic_fdr, cyclic_lon = add_cyclic_point(fdr, coord=ffdi_lon)
    ax[0,1].pcolor(cyclic_lon, ffdi_lat, cyclic_fdr, hatch='//////', shading='auto', alpha=0, rasterized=False)

    # ================ FFDI change point year
    years = ffdi_pettitt_all.cp_year.where(ffdi_pettitt_all.cp_year > fy).values
    
    ax[1,0].add_feature(cartopy.feature.LAND, color=bad_col, zorder=0) # Ensures masked out areas are bad_col
    cyclic_years, cyclic_lon = add_cyclic_point(years, coord=ffdi_lon)
    p_ffdi_cp_y = ax[1,0].pcolormesh(cyclic_lon, ffdi_lat, cyclic_years, cmap=yr_cmap, zorder=1, vmin=fy, vmax=2020, rasterized=True)
    
    # ================ FFDI change point magnitude    
    mags = xr.where(ffdi_pettitt_all.cp_magnitude.isnull(), -999, ffdi_pettitt_all.cp_magnitude).values
    cyclic_mags, cyclic_lon = add_cyclic_point(mags, coord=ffdi_lon)
    p_ffdi_cp_m = ax[1,1].pcolormesh(cyclic_lon, ffdi_lat, cyclic_mags, cmap=ffdi_cmap, zorder=1, vmin=-60, vmax=60, rasterized=True)
    
    fdr = ffdi_pettitt_all.fdr_10
    fdr = xr.where(fdr == 1, 1, np.nan).values
    cyclic_fdr, cyclic_lon = add_cyclic_point(fdr, coord=ffdi_lon)
    ax[1,1].pcolor(cyclic_lon, ffdi_lat, cyclic_fdr, hatch='//////', shading='auto', alpha=0, rasterized=False)

    for a, ax_label in zip(ax.flatten(), [r'$\bf{a}$', r'$\bf{b}$', r'$\bf{c}$', r'$\bf{d}$']):
        a.coastlines(lw=coastlines_lw, zorder=6)
        a.set_extent([-179.99, 180, 90, -60])
        a.add_feature(cartopy.feature.OCEAN, color='w', zorder=4)
        for k, spine in a.spines.items():  #ax.spines is a dictionary
                spine.set_zorder(10)
        a.text(0.01, 0.98, ax_label,
            ha='left', va='top', transform=a.transAxes, zorder=12)
        
    cb_ax = fig.add_axes([0.111, 0.52, 0.01, 0.34])
    cb = fig.colorbar(p_pr_t, cax=cb_ax, orientation='vertical', ticks=np.arange(-0.05, 0.051, 0.025), extend='neither')
    cb.ax.yaxis.set_ticks_position('left')
    cb.ax.set_ylabel(r"$P_a ' / \sigma_{P_a}$ trend [$\sigma$ y$^{-1}$]")
    cb.ax.yaxis.set_label_position('left')
    
    cb_ax = fig.add_axes([0.905, 0.52, 0.01, 0.34])
    cb = fig.colorbar(p_ffdi_t, cax=cb_ax, orientation='vertical', ticks=np.arange(-0.9, 0.91, 0.3))
    cb.ax.set_ylabel(r'$\mathrm{FWD}_f$ trend [d y$^{-1}$]')
    
    cb_ax = fig.add_axes([0.111, 0.135, 0.01, 0.34])
    cb = fig.colorbar(p_ffdi_cp_y, cax=cb_ax, orientation='vertical')
    cb.ax.yaxis.set_ticks(range(fy, 2021, 10))
    cb.ax.yaxis.set_ticks_position('left')
    cb.ax.set_ylabel('$\mathrm{FWD}_f$ step change\nyear')
    cb.ax.yaxis.set_label_position('left')
    
    cb_ax = fig.add_axes([0.905, 0.135, 0.01, 0.34])
    cb = fig.colorbar(p_ffdi_cp_m, cax=cb_ax, orientation='vertical', ticks=np.arange(-60, 61, 30))
    cb.ax.set_ylabel('$\mathrm{FWD}_f$ step change\nmagnitude '+r'[d y$^{-1}$]')
        
    plt.subplots_adjust(hspace=0.015, wspace=0.02)
    
    plt.savefig('./figures/ffdi_precip_mk_pettitt_'+str(fy)+'-2020.pdf', format='pdf', dpi=400, bbox_inches='tight')

# Figure S3: Precipitation Pettitt test

In [ ]:
figsize = (6.9,1.5)

# Colormap stuff
rgb_val = 0.9
bad_col = np.repeat(rgb_val, 3)

# ffdi_cmap = copy.copy(custom_cmap)
# ffdi_cmap.set_under(bad_col)

pr_cmap = copy.copy(mpl.cm.get_cmap('BrBG'))
pr_cmap.set_under(bad_col)

# For change point years, use discrete colormap
# select desired starting colours
colours = np.asarray(plt.rcParams['axes.prop_cycle'].by_key()['color'])[[7,4,0,1,2,6]]
# get three grads of lightness
l_colours = [fn.adjust_lightness(c, a) for c in colours for a in [1.3, 0.9]]
cmap = mpl.colors.ListedColormap(l_colours)
norm = mpl.colors.BoundaryNorm(np.arange(12.01), cmap.N) 
idx = [11,0,1,2,3,6,7,4,5,9]
l_colours = [l_colours[lc] for lc in idx]
yr_cmap = mpl.colors.ListedColormap(l_colours)

with mpl.rc_context(plt_params):
    ffdi_lon = ffdi_pettitt_all.lon
    ffdi_lat = ffdi_pettitt_all.lat
    
    gpcc_lon = gpcc_pettitt_all.lon
    gpcc_lat = gpcc_pettitt_all.lat
    
    fig, ax = plt.subplots(1, 2, figsize=figsize, subplot_kw={'projection': ccrs.PlateCarree()})

    # ================ Precip change point year
    years = gpcc_pettitt_all.cp_year.where(gpcc_pettitt_all.cp_year > fy).values
    
    ax[0].add_feature(cartopy.feature.LAND, color=bad_col, zorder=0) # Ensures masked out areas are bad_col
    cyclic_years, cyclic_lon = add_cyclic_point(years, coord=gpcc_lon)
    p_gpcc_cp_y = ax[0].pcolormesh(cyclic_lon, gpcc_lat, cyclic_years, cmap=yr_cmap, zorder=1, vmin=fy, vmax=2020, rasterized=True)
    
    # ================ Precip change point magnitude    
    mags = xr.where(gpcc_pettitt_all.cp_magnitude.isnull(), -999, gpcc_pettitt_all.cp_magnitude).values
    cyclic_mags, cyclic_lon = add_cyclic_point(mags, coord=gpcc_lon)
    p_gpcc_cp_m = ax[1].pcolormesh(cyclic_lon, gpcc_lat, cyclic_mags, cmap=pr_cmap, zorder=1, vmin=-3.2, vmax=3.2, rasterized=True)
    
    fdr = gpcc_pettitt_all.fdr_10
    fdr = xr.where(fdr == 1, 1, np.nan).values
    cyclic_fdr, cyclic_lon = add_cyclic_point(fdr, coord=gpcc_lon)
    ax[1].pcolor(cyclic_lon, gpcc_lat, cyclic_fdr, hatch='//////', shading='auto', alpha=0, rasterized=False)

    for a, ax_label in zip(ax, [r'$\bf{a}$', r'$\bf{b}$']):
        a.coastlines(lw=coastlines_lw, zorder=6)
        a.set_extent([-179.99, 180, 90, -60])
        a.add_feature(cartopy.feature.OCEAN, color='w', zorder=4)
        for k, spine in a.spines.items():  #ax.spines is a dictionary
                spine.set_zorder(10)
        a.text(0.01, 0.98, ax_label,
            ha='left', va='top', transform=a.transAxes, zorder=12)
    
    cb_ax = fig.add_axes([0.111, 0.135, 0.01, 0.72])
    cb = fig.colorbar(p_gpcc_cp_y, cax=cb_ax, orientation='vertical')
    cb.ax.yaxis.set_ticks(range(fy, 2021, 10))
    cb.ax.yaxis.set_ticks_position('left')
    cb.ax.set_ylabel(r"$P_a ' / \sigma_{P_a}$" + ' step change\nyear')
    cb.ax.yaxis.set_label_position('left')
    
    cb_ax = fig.add_axes([0.905, 0.135, 0.01, 0.72])
    cb = fig.colorbar(p_gpcc_cp_m, cax=cb_ax, orientation='vertical', ticks=np.arange(-3.2, 3.3, 0.8))
    cb.ax.set_ylabel(r"$P_a ' / \sigma_{P_a}$" +' step change\nmagnitude '+r'[$\sigma$ y$^{-1}$]')
        
    plt.subplots_adjust(hspace=0.015, wspace=0.02)
    
    plt.savefig('./figures/precip_pettitt_'+str(fy)+'-2020.pdf', format='pdf', dpi=400, bbox_inches='tight')

# Trends in temperature, relative humidity and wind speed

#### Hack some functions from `functions.py` to speed things up

In [ ]:
def theil_sen_1d(X):
    """Performs the Mann-Kendall trend test and calculates the Theil-Sen slope"""
    
    from scipy.stats import theilslopes
    
    X = X[~np.isnan(X)]    
    ts_slope, ts_intercept = theilslopes(X)[:2] # First element is estimate of slope, second is intercept
    
    return (ts_slope, ts_intercept)

def theil_sen_nan_wrap(X):
    """
        Only run Mann-Kendall test on data that isn't just NaNs.
    """
    if np.count_nonzero(~np.isnan(X)) > 0:
        return theil_sen_1d(X)
    else:
        return (np.nan, np.nan)


def apply_theil_sen(da):
    """
        Applies Mann-Kendall test to every grid box in da.
        Also calculates the Theil-Sen slope and intercept.
        
        Parameters
        ----------
        da : xarray DataArray
            Array of values.
            
        Returns
        -------
        ds : xarray DataSet
            DataSet with the variables S, ts_slope and ts_intercept.
    """
    ts = xr.apply_ufunc(theil_sen_nan_wrap, da, input_core_dims=[['time']], output_core_dims=[[], []], dask='allowed', vectorize=True)
    ds = ts[0].to_dataset(name='ts_slope').merge(ts[1].to_dataset(name='ts_intercept'))
    
    return ds

#### Relative humidity, maximum temperature and maximum wind speed

In [ ]:
jra_rh = xr.open_zarr('/scratch1/ric368/projects/fire/data/jra_daily_rh_global_19580101_20201231.zarr',
                      consolidated=True)
jra_rh = jra_rh.rename({'RH_GDS0_HTGL': 'rh'}).rh.sel(time=slice(str(fy), str(ly)))
jra_rh = jra_rh.chunk({'time': -1, 'lat': 30, 'lon': 50})

In [ ]:
jra_tmax = xr.open_zarr('/scratch1/ric368/projects/fire/data/jra_daily_tmax_global_19580101_20201231.zarr',
                          consolidated=True)
jra_tmax = jra_tmax.rename({'TMAX_GDS4_HTGL': 'tmax'}).tmax
jra_tmax = jra_tmax.chunk({'time': -1, 'lat': 30, 'lon': 50})

In [ ]:
jra_wmax = xr.open_zarr('/scratch1/ric368/projects/fire/data/jra_daily_wmax_global_19580101_20201231.zarr',
                          consolidated=True)
jra_wmax = jra_wmax.rename({'WSMX_GDS4_HTGL': 'wmax'}).wmax
jra_wmax = jra_wmax.chunk({'time': -1, 'lat': 30, 'lon': 50})

#### Annual averages, where the year is the fire year

In [ ]:
year_ends = xr.open_zarr('/scratch1/ric368/projects/fire/data/derived/jra_ffdi_fire_year_end_month.zarr',
                         consolidated=True).month
year_ends = year_ends.compute()

In [ ]:
%%time
# Aggregate to monthly and compute (~100MB per array)
rh_m = jra_rh.resample({'time': '1MS'}).mean().compute()
tmax_m = jra_tmax.resample({'time': '1MS'}).mean().compute()
wmax_m = jra_wmax.resample({'time': '1MS'}).mean().compute()

In [ ]:
# Switch lons to -180-180
rh_m = fn.switch_lons(rh_m)
tmax_m = fn.switch_lons(tmax_m)
wmax_m = fn.switch_lons(wmax_m)

In [ ]:
# Can use fn.accumulate_to_year_end. We want 5 months centred on max FFDI month, so need to accumulate over 5 months and shift data forward 3 months
# e.g. for a location where the max FFDI month is July, and hence year end is Dec, we want the data from Apr-Sep, so we need to shift this forwards 3 months
shift = 3
acc = 5
rh_ann = fn.accumulate_to_year_end(rh_m, year_ends, shift=shift, accumulate=acc) / acc
tmax_ann = fn.accumulate_to_year_end(tmax_m, year_ends, shift=shift, accumulate=acc) / acc
wmax_ann = fn.accumulate_to_year_end(wmax_m, year_ends, shift=shift, accumulate=acc) / acc

In [ ]:
%%time
rh_ts = apply_theil_sen(rh_ann.where(jra_mask == 1))
tmax_ts = apply_theil_sen(tmax_ann.where(jra_mask == 1))
wmax_ts = apply_theil_sen(wmax_ann.where(jra_mask == 1))

In [ ]:
# Plot data
tmax_p = tmax_ts.ts_slope.sel(lat=slice(90,-60))
rh_p = rh_ts.ts_slope.sel(lat=slice(90,-60))
wmax_p = wmax_ts.ts_slope.sel(lat=slice(90,-60))

In [ ]:
# Get max and min to choose limits
print(tmax_p.min().values, tmax_p.max().values)
print(rh_p.min().values, rh_p.max().values)
print(wmax_p.min().values, wmax_p.max().values)

# Figure S4: FFDI components trend

In [ ]:
with mpl.rc_context(plt_params):
    
    fig, ax = plt.subplots(2, 2, figsize=(6.9, 3), subplot_kw={'projection': ccrs.PlateCarree()})

    # ================ Precip trend    
    p1 = tmax_p.plot(ax=ax[0,0], cmap='RdBu_r', vmin=-0.12, vmax=0.12, rasterized=True, add_colorbar=False)
    p2 = rh_p.plot(ax=ax[0,1], cmap='BrBG', vmin=-0.4, vmax=0.4, rasterized=True, add_colorbar=False)
    p3 = wmax_p.plot(ax=ax[1,0], cmap='RdBu_r', vmin=-0.06, vmax=0.06, rasterized=True, add_colorbar=False)
    
    cb_ax = fig.add_axes([0.111, 0.52, 0.01, 0.34])
    cb = fig.colorbar(p1, cax=cb_ax, orientation='vertical', ticks=np.arange(-0.24, 0.25, 0.06), extend='max')
    cb.ax.yaxis.set_ticks_position('left')
    cb.ax.set_ylabel("Maximum daily\ntemperature "+r"[$^{\circ}$C y$^{-1}$]")
    cb.ax.yaxis.set_label_position('left')
    
    cb_ax = fig.add_axes([0.905, 0.52, 0.01, 0.34])
    cb = fig.colorbar(p2, cax=cb_ax, orientation='vertical', ticks=np.arange(-0.4, 0.41, 0.2), extend='min')
    cb.ax.set_ylabel("Daily relative\nhumidity "+r"[% y$^{-1}$]")
    
    cb_ax = fig.add_axes([0.111, 0.135, 0.01, 0.34])
    cb = fig.colorbar(p3, cax=cb_ax, orientation='vertical', ticks=np.arange(-0.06, 0.061, 0.02), extend='both')
    cb.ax.yaxis.set_ticks_position('left')
    cb.ax.set_ylabel("Maximum daily wind\nspeed "+r"[m s$^{-1}$ y$^{-1}$]")
    cb.ax.yaxis.set_label_position('left')

    p4 = ffdi_mk_ds.sel(lat=slice(90,-60)).sum('L').ts_slope.plot(ax=ax[1,1], cmap=custom_cmap, vmin=-1, vmax=1, rasterized=True, add_colorbar=False)
    ax[1,1].set_title('')
    
    cb_ax = fig.add_axes([0.905, 0.135, 0.01, 0.34])
    cb = fig.colorbar(p4, cax=cb_ax, orientation='vertical', ticks=np.arange(-1, 1.1, 0.5))
    cb.ax.set_ylabel(r'$\mathrm{FWD}_f$ [d y$^{-1}$]')
    
    for a, ax_label in zip(ax.flatten(), [r'$\bf{a}$', r'$\bf{b}$', r'$\bf{c}$', r'$\bf{d}$']):
        a.coastlines(lw=coastlines_lw, zorder=6)
        a.text(0.01, 0.98, ax_label,
            ha='left', va='top', transform=a.transAxes, zorder=12)
        
    plt.subplots_adjust(hspace=0.015, wspace=0.02)
    
    plt.savefig('./figures/ffdi_components_trend_'+str(fy)+'-2020.pdf', format='pdf', dpi=400, bbox_inches='tight')

# WCRP/BAMS Figure: FFDI trend

In [ ]:
figsize = (6.9,3.)

# Colormap stuff
rgb_val = 0.9
bad_col = np.repeat(rgb_val, 3)

ffdi_cmap = copy.copy(mpl.cm.get_cmap('RdBu_r'))
ffdi_cmap.set_under(bad_col)

with mpl.rc_context(plt_params):
    ffdi_lon = ffdi_mk_all.lon
    ffdi_lat = ffdi_mk_all.lat
    
    fig, ax = plt.subplots(1, 1, figsize=figsize, subplot_kw={'projection': ccrs.Robinson()})

    # ================ FFDI trend
    mags = xr.where(ffdi_mk_all.ts_slope.isnull(), -999, ffdi_mk_all.ts_slope).values
    cyclic_mags, cyclic_lon = add_cyclic_point(mags, coord=ffdi_lon)
    p_ffdi_t = ax.pcolormesh(cyclic_lon, ffdi_lat, cyclic_mags, cmap=ffdi_cmap, zorder=1, vmin=-0.9, vmax=0.9, rasterized=True,
                            transform=ccrs.PlateCarree())
    
    fdr = ffdi_mk_all.fdr_10
    fdr = xr.where(fdr == 1, 1, np.nan).values
    cyclic_fdr, cyclic_lon = add_cyclic_point(fdr, coord=ffdi_lon)
    ax.pcolor(cyclic_lon, ffdi_lat, cyclic_fdr, hatch='......', shading='auto', alpha=0, rasterized=False,
             transform=ccrs.PlateCarree())

    ax.set_global()
    ax.coastlines(lw=coastlines_lw, zorder=6)
    ax.add_feature(cartopy.feature.OCEAN, color='w', zorder=4)
    for k, spine in ax.spines.items():  #ax.spines is a dictionary
        spine.set_zorder(10)
    
    cb_ax = fig.add_axes([0.85, 0.13, 0.015, 0.7])
    cb = fig.colorbar(p_ffdi_t, cax=cb_ax, orientation='vertical', ticks=np.arange(-0.9, 0.91, 0.3))
    cb.ax.set_ylabel(r'95th percentile FFDI trend [d y$^{-1}$]')
            
    plt.savefig('./figures/wcrp_bams_ffdi_mk_'+str(fy)+'-2020.pdf', format='pdf', dpi=400, bbox_inches='tight')

# Close cluster

In [ ]:
client.close()
cluster.close()